In [1]:
from ns import ns

[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { _ZN3ns3L16g_timeInitHelperE, _GLOBAL__sub_I_cling_module_143, __orc_init_func.cling-module-143, __cxx_global_var_initcling_module_143_, $.cling-module-143.__inits.0 }) }
[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { __orc_init_func.cling-module-143 }) }


In [2]:
# creating nodes

all_nodes = ns.NodeContainer()
all_nodes.Create(7)

In [3]:
all_nodes.GetN()

7

In [4]:
# creating Links / Connections

csma = ns.CsmaHelper()
switch_net_devices = ns.NetDeviceContainer() # vsechny sitovky od Switche
net_net_devices = ns.NetDeviceContainer() # vsechny sitovky z prostredni site 

In [5]:
# Router 1 (left)

csma_node_pair = ns.NodeContainer()
csma_node_pair.Add(all_nodes.Get(1)) # beru Router 1
csma_node_pair.Add(all_nodes.Get(2)) # beru Switch
devices = csma.Install(csma_node_pair) # spojim je kabelem


net_net_devices.Add(devices.Get(0)) # Pridam sitovku do prostredni site
switch_net_devices.Add(devices.Get(1)) # Pridam sitovku do switche


# Router 2 (right)

csma_node_pair = ns.NodeContainer()
csma_node_pair.Add(all_nodes.Get(3)) # beru Router 2
csma_node_pair.Add(all_nodes.Get(2)) # beru Switch
devices = csma.Install(csma_node_pair) # spojim je kabelem


net_net_devices.Add(devices.Get(0)) # Pridam sitovku do prostredni site
switch_net_devices.Add(devices.Get(1)) # Pridam sitovku do switche


# PC3
csma_node_pair = ns.NodeContainer()
csma_node_pair.Add(all_nodes.Get(5)) # beru PC3
csma_node_pair.Add(all_nodes.Get(2)) # beru Switch
devices = csma.Install(csma_node_pair) # spojim je kabelem


net_net_devices.Add(devices.Get(0)) # Pridam sitovku do prostredni site
switch_net_devices.Add(devices.Get(1)) # Pridam sitovku do switche

# PC4
csma_node_pair = ns.NodeContainer()
csma_node_pair.Add(all_nodes.Get(6)) # beru PC4
csma_node_pair.Add(all_nodes.Get(2)) # beru Switch
devices = csma.Install(csma_node_pair) # spojim je kabelem

net_net_devices.Add(devices.Get(0)) # Pridam sitovku do prostredni site
switch_net_devices.Add(devices.Get(1)) # Pridam sitovku do switche

In [6]:
# P2P spoj mezi PC1 a Router 1

router_node_container = ns.NodeContainer()
router_node_container.Add(all_nodes.Get(1)) # Pridam Router 1
router_node_container.Add(all_nodes.Get(0)) # Pridam PC1
p2p = ns.PointToPointHelper() # Vytvorim genericky P2P spoj
p2p_left = p2p.Install(router_node_container) # Priradim ho P2P spoji vlevo


# P2P spoj mezi PC1 a Router 1
router_node_container = ns.NodeContainer()
router_node_container.Add(all_nodes.Get(3)) # Pridam Router 2
router_node_container.Add(all_nodes.Get(4)) # Pridam PC2
p2p = ns.PointToPointHelper() # Vytvorim genericky P2P spoj
p2p_right = p2p.Install(router_node_container) # Priradim ho P2P spoji vpravo

In [7]:
# creating switch
bridge_helper = ns.BridgeHelper()
bridge_helper.Install(all_nodes.Get(2), switch_net_devices) # vsechny sitovky Switche dam dokupy a vytvorim switch

In [8]:
# IP Stack - instalace IP protokolu na vsechny zarizeni

internet = ns.InternetStackHelper()
internet.Install(all_nodes.Get(0))
internet.Install(all_nodes.Get(1))
internet.Install(all_nodes.Get(3))
internet.Install(all_nodes.Get(4))
internet.Install(all_nodes.Get(5))
internet.Install(all_nodes.Get(6))

In [9]:
# IP addresace
addr = ns.Ipv4AddressHelper()
addr.SetBase(ns.Ipv4Address(f"10.0.1.0"), ns.Ipv4Mask("255.255.255.0"))
device_ifaces = addr.Assign(net_net_devices) # Priradi IP addresy pro vsechny interfacy pro Sit1

addr.NewNetwork() # Vytvori novou sit, na zaklade te predchozi (pouzije stejnou masku)
device_ifaces = addr.Assign(p2p_left) # Priradi IP addresy pro vsechny interfacy pro P2P spoj

addr.NewNetwork() # Vytvori novou sit, na zaklade te predchozi (pouzije stejnou masku)
device_ifaces = addr.Assign(p2p_right) # Priradi IP addresy pro vsechny interfacy pro P2P spoj

In [10]:
# Vytvori routovaci tabulku na vsech L3 zarizenich
ns.Ipv4GlobalRoutingHelper.PopulateRoutingTables() 

In [11]:
# Vytvareni serveru RED (ktery bude naslouchat na portu 11 a odpovidat)

echo_srv_helper = ns.UdpEchoServerHelper(11) #vytvorim si helper, neboli genericky server
srv_apps_red = echo_srv_helper.Install(all_nodes.Get(5)) # Nainstaluju ho na PC2
print(f"Servers: {srv_apps_red.GetN()}")

# Vytvareni serveru GREEN (ktery bude naslouchat na portu 22 a odpovidat)

echo_srv_helper = ns.UdpEchoServerHelper(22) #vytvorim si helper, neboli genericky server
srv_apps_green = echo_srv_helper.Install(all_nodes.Get(6)) # Nainstaluju ho na PC2
print(f"Servers: {srv_apps_green.GetN()}")

Servers: 1
Servers: 1


In [12]:
# Vytvareni clientu RED

# Na dalsich radcich se vytvori genericky client
# Prvni radek vytvori generickeho klienta, ktery bude "pingovat" server 10.0.1.3 na portu 11
echo_client_helper = ns.UdpEchoClientHelper(ns.Address(ns.InetSocketAddress(ns.Ipv4Address("10.0.1.3"),11).ConvertTo()))
# Na dalsich radcich vytvarime atributy pro klienty, kolik packetu, jak casto, jak velke
echo_client_helper.SetAttribute("MaxPackets", ns.UintegerValue(50))
echo_client_helper.SetAttribute("Interval", ns.TimeValue(ns.Seconds(1)))
echo_client_helper.SetAttribute("PacketSize", ns.UintegerValue(1500))

client_node_container = ns.NodeContainer() # vytvoreni kontejneru pro vsechny clienty
client_node_container.Add(all_nodes.Get(0)) # pridani PC1 do kontejneru

client_apps_red = echo_client_helper.Install(client_node_container) # pridani generickeho klienta do vsech zarizeni v kontejneru


# Vytvareni clientu GREEN

# Na dalsich radcich se vytvori genericky client
# Prvni radek vytvori generickeho klienta, ktery bude "pingovat" server 10.0.1.4 na portu 22
echo_client_helper = ns.UdpEchoClientHelper(ns.Address(ns.InetSocketAddress(ns.Ipv4Address("10.0.1.4"),22).ConvertTo()))
# Na dalsich radcich vytvarime atributy pro klienty, kolik packetu, jak casto, jak velke
echo_client_helper.SetAttribute("MaxPackets", ns.UintegerValue(50))
echo_client_helper.SetAttribute("Interval", ns.TimeValue(ns.Seconds(1)))
echo_client_helper.SetAttribute("PacketSize", ns.UintegerValue(1500))

client_node_container = ns.NodeContainer() # vytvoreni kontejneru pro vsechny clienty
client_node_container.Add(all_nodes.Get(4)) # pridani PC2 do kontejneru

client_apps_green = echo_client_helper.Install(client_node_container) # pridani generickeho klienta do vsech zarizeni v kontejneru

In [13]:
# Logging

ns.LogComponentEnable("UdpEchoClientApplication", ns.LOG_LEVEL_INFO)
ns.LogComponentEnable("UdpEchoServerApplication", ns.LOG_LEVEL_INFO)

In [14]:
# Timing

srv_apps_red.Start(ns.Seconds(1.0))
srv_apps_red.Stop(ns.Seconds(10.0))
client_apps_red.Start(ns.Seconds(2.0)) # has to be later than server
client_apps_red.Stop(ns.Seconds(10.0))


srv_apps_green.Start(ns.Seconds(11.0))
srv_apps_green.Stop(ns.Seconds(20.0))
client_apps_green.Start(ns.Seconds(12.0)) # has to be later than server
client_apps_green.Stop(ns.Seconds(20.0))

In [15]:
# Run

ns.Simulator.Stop(ns.Seconds(35))
ns.Simulator.Run()
ns.Simulator.Destroy()


At time +2s client sent 1500 bytes to 10.0.1.3 port 11
At time +2.37891s server received 1500 bytes from 10.0.2.2 port 49153
At time +2.37891s server sent 1500 bytes to 10.0.2.2 port 49153
At time +2.75982s client received 1500 bytes from 10.0.1.3 port 11
At time +3s client sent 1500 bytes to 10.0.1.3 port 11
At time +3.37891s server received 1500 bytes from 10.0.2.2 port 49153
At time +3.37891s server sent 1500 bytes to 10.0.2.2 port 49153
At time +3.75782s client received 1500 bytes from 10.0.1.3 port 11
At time +4s client sent 1500 bytes to 10.0.1.3 port 11
At time +4.37891s server received 1500 bytes from 10.0.2.2 port 49153
At time +4.37891s server sent 1500 bytes to 10.0.2.2 port 49153
At time +4.75782s client received 1500 bytes from 10.0.1.3 port 11
At time +5s client sent 1500 bytes to 10.0.1.3 port 11
At time +5.37891s server received 1500 bytes from 10.0.2.2 port 49153
At time +5.37891s server sent 1500 bytes to 10.0.2.2 port 49153
At time +5.75782s client received 1500 byte